In [1]:
import lumipy as lm
from lusidjam import RefreshingToken as rt

atlas = lm.get_atlas(token=rt())

In [2]:
portfolios = atlas.lusid_portfolio()

pf_codes = portfolios.select(
    portfolios.portfolio_code
).where(
    portfolios.portfolio_scope == 'Finbourne-Examples'
).go().PortfolioCode.tolist()

pf_codes

['US-Corporate-Bond',
 'Global-Equity',
 'UK-Equities',
 'US-Treasury-Bond',
 'UK-Swaps-aggressive']

# Tutorial 6 - Unions

Unions are set operations that act on table rows. `UNION` of two subqueries is the set of rows from both with duplicates removed (`.union()`). `UNION ALL` is the same thing but keeps the duplicate rows (`.union_all()`).

In this tutorial we'll build up a more complicated query where we use `.union()` concatenate a collection of table variable subqueries each of which depend on a scalar variable. Each subquery will get the top five instruments in a portfolio code by `CostAmountPortfolioCurrency`.  

In [3]:
holding = atlas.lusid_portfolio_holding()

In [4]:
def subquery(pf_code):

    total_cost = holding.select(
        TotalCost=holding.cost_amount_portfolio_currency.sum()
    ).where(
        (holding.portfolio_code == pf_code) & 
        (holding.portfolio_scope == 'Finbourne-Examples')
    ).to_scalar_var()

    cost_table = holding.select(
        holding.portfolio_code,
        holding.lusid_instrument_id, 
        holding.cost_amount_portfolio_currency,
        CostFractionPc=100*holding.cost_amount_portfolio_currency/total_cost
    ).where(
        (holding.portfolio_code == pf_code) & 
        (holding.portfolio_scope == 'Finbourne-Examples')
    ).order_by(
        (holding.cost_amount_portfolio_currency/total_cost).descending()
    ).limit(
        5
    ).to_table_var()
    
    return cost_table.select('*')

subqueries = [subquery(c) for c in pf_codes]

## Union of Two Subqueries
You can take the union of two subqueries by using the `union()` method on another subquery (not containing `group by`, `order by` or `limit`). If the `union()` method is not available consider turning it into a table variable with `to_table_variable()`, calling `select('*')` and then calling `union()` on that. 

In [5]:
sq1, sq2 = subqueries[:2]
qry = sq1.union(sq2)
df = qry.go()
df

,PortfolioCode,LusidInstrumentId,CostAmountPortfolioCurrency,CostFractionPc
0,Global-Equity,LUID_1W5WYN3O,1.760183e+09,20.114573
1,Global-Equity,LUID_4CINGMZM,1.322168e+09,15.109133
2,Global-Equity,LUID_GJ9TICS4,1.050759e+09,12.007598
3,Global-Equity,LUID_J4G8D0DV,1.146539e+09,13.102119
4,Global-Equity,LUID_JTQY6QFI,1.822715e+09,20.829161
5,US-Corporate-Bond,LUID_0Y2HFZFM,2.069485e+07,20.978052
6,US-Corporate-Bond,LUID_5TCI1RPB,6.935512e+07,70.304215
7,US-Corporate-Bond,LUID_9524ZNMO,1.689421e+07,17.125399
8,US-Corporate-Bond,LUID_RV9XYISA,1.257152e+07,12.743559
9,US-Corporate-Bond,LUID_XB11B4VW,1.134238e+08,114.975923


## Union Over a Subquery Collection
The `lumipy.concat()` convenience function at the top of the lumipy library can be used to union a whole collection of subqueries by chaining the unions for you. 

In [6]:
qry = lm.concat(subqueries)

In [7]:
df = qry.go()
df

,PortfolioCode,LusidInstrumentId,CostAmountPortfolioCurrency,CostFractionPc
0,Global-Equity,LUID_1W5WYN3O,1.760183e+09,20.114573
1,Global-Equity,LUID_4CINGMZM,1.322168e+09,15.109133
2,Global-Equity,LUID_GJ9TICS4,1.050759e+09,12.007598
3,Global-Equity,LUID_J4G8D0DV,1.146539e+09,13.102119
4,Global-Equity,LUID_JTQY6QFI,1.822715e+09,20.829161
5,UK-Equities,LUID_941RZG0H,2.871400e+05,2.965447
6,UK-Equities,LUID_ARH76A7B,2.314375e+05,2.390177
7,UK-Equities,LUID_E9EW4TBO,3.868000e+05,3.994688
8,UK-Equities,LUID_KJFBD4GQ,5.552418e+06,57.342760
9,UK-Equities,LUID_LH6UUF74,2.585520e+06,26.702035


We can observe that the concise code above produces a lengthy SQL query string.

In [8]:
# Print the underlying SQL
qry.print_sql()

@@sv_01588649365467531 = 
select
  total([CostAmountPortfolioCurrency]) as [TotalCost] 
from
  Lusid.Portfolio.Holding 
where
  ([PortfolioCode] = 'US-Corporate-Bond') and ([PortfolioScope] = 'Finbourne-Examples');

@@sv_99850381898748780 = 
select
  total([CostAmountPortfolioCurrency]) as [TotalCost] 
from
  Lusid.Portfolio.Holding 
where
  ([PortfolioCode] = 'Global-Equity') and ([PortfolioScope] = 'Finbourne-Examples');

@@sv_04777894227422584 = 
select
  total([CostAmountPortfolioCurrency]) as [TotalCost] 
from
  Lusid.Portfolio.Holding 
where
  ([PortfolioCode] = 'UK-Equities') and ([PortfolioScope] = 'Finbourne-Examples');

@@sv_103301083146014988 = 
select
  total([CostAmountPortfolioCurrency]) as [TotalCost] 
from
  Lusid.Portfolio.Holding 
where
  ([PortfolioCode] = 'US-Treasury-Bond') and ([PortfolioScope] = 'Finbourne-Examples');

@@sv_62597977708324301 = 
select
  total([CostAmountPortfolioCurrency]) as [TotalCost] 
from
  Lusid.Portfolio.Holding 
where
  ([PortfolioCode] =

## Union Types

There are two types of union available: `union` (like above) and `union all` both will concatenate tables of data together along the column axis but the former will filter duplicate rows.